## DR.DECR (Dense Retrieval with Distillation-Enhanced Cross-Lingual Representation) Student/Teacher Training
In this notebook, we show how to train a model using Knowledge Distillation (Student/Teacher) to improve performance on Cross-lingual retrieval, as desribed in `Learning Cross-Lingual IR from an English Retriever`, https://arxiv.org/abs/2112.08185.

After training the model, we index the data and run search.  


In orded to run (almost) instantaneously, we use trivial sizes of training data and collection to search.

## Dependencies
If not already done, please make sure to install PrimeQA with notebooks extras before getting started.

In [1]:
# If you want CUDA 11 uncomment and run this (for CUDA 10 or CPU you can ignore this line).
#! pip install 'torch~=1.11.0' --extra-index-url https://download.pytorch.org/whl/cu113

# Uncomment to install PrimeQA from source (pypi package pending).
# The path should be the project root (e.g. '.' below).
#! pip install .[notebooks]

## Configuration
First, we need to include the required modules.

In [2]:
import os
import tempfile
 
from primeqa.ir.dense.colbert_top.colbert.utils.utils import create_directory, print_message
from primeqa.ir.dense.colbert_top.colbert.infra import Run, RunConfig
from primeqa.ir.dense.colbert_top.colbert.infra.config import ColBERTConfig
from primeqa.ir.dense.colbert_top.colbert.training.training import train
from primeqa.ir.dense.colbert_top.colbert.indexing.collection_indexer import encode
from primeqa.ir.dense.colbert_top.colbert.searcher import Searcher

No CUDA runtime is found, using CUDA_HOME='/opt/share/cuda-11.8'
{"time":"2023-07-05 01:19:59,809", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss."}
{"time":"2023-07-05 01:19:59,842", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss."}


## Training
We will train a ColBERT model using a TSV file containing [query, positive document, negative document] triples. We use the XOR-TyDi dataset, as described here: https://nlp.cs.washington.edu/xorqa/

The path in `test_files_location` below points to the location of files used by the notebook, by default it points to the files used by CI testing.

In [3]:
test_files_location = '../../../tests/resources/ir_dense'
model_name_or_path = 'xlm-roberta-base'
model_type = 'xlm-roberta'
with tempfile.TemporaryDirectory() as working_dir:
    output_dir=os.path.join(working_dir, 'output_dir')
text_triples_fn = os.path.join(test_files_location, "xorqa.train_ir_negs_5_poss_1_001pct_at_0pct.tsv")

Here is an example of a training file record:

In [4]:
import pandas as pd
from IPython.display import display, HTML
data = pd.read_csv(text_triples_fn, sep='\t', nrows=1, header=None)
display(HTML(data.to_html()))

,0,1,2
0,중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?,"Kangxi Emperor The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang.","Chiddy Bang new songs from the duo and in November 2009 debuted the group's first free mixtape entitled ""The Swelly Express"". On 28 April 2011 during the first-ever MTV O Music Awards, Anamege broke the Guinness World Record for Longest Freestyle Rap and Longest Marathon Rapping Record by freestyling for 9 hours, 18 minutes, and 22 seconds, stealing the throne from rapper M-Eighty, who originally broke the record in 2009 by rapping for 9 hours, 15 minutes and 15 seconds. Anamege had also beat Canadian rapper D.O. for Longest Marathon Rapping session, the previous record being for 8 hours and 45 minutes."


In [5]:
args_dict = {
                'root': output_dir,
                'experiment': 'test_training_student',
                'triples': text_triples_fn,
                'model_type': model_type,
                'checkpoint': model_name_or_path,
                'maxsteps': 3,
                'bsize': 1,
                'accumsteps': 1,
                'amp': True,
                'epochs': 1,
                'rank': 0,
                'nranks': 1
            }

Next we train the the student starting-point model, and save it's location in the `student_model_fn`variable

In [6]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)  
    student_model_fn = train(colBERTConfig, text_triples_fn, None, None)

{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "cosine",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta",
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta",
    "teacher_doc_maxlen": 180,
    "disti

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Jul 05, 01:20:24] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jul 05, 01:20:24] 1 epochs of 5 examples
[Jul 05, 01:20:24] batch size: 1
[Jul 05, 01:20:24] maxsteps set to 3
[Jul 05, 01:20:24] start batch idx: 0
[Jul 05, 01:20:24] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Jul 05, 01:20:24] #> Input: $ 중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?, 		 True, 		 None
[Jul 05, 01:20:24] #> Output IDs: torch.Size([32]), tensor([     0,   9748,  24120,   1180,  13968, 211059,  83639,  76826,  78363,
         57104,    993, 161732,    697, 116932, 114150,     32,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[Jul 05, 01:20:24] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[Jul 05, 01:20:24] #> XLMR DocTokenizer.te

/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 01:20:25] #>>>> before linear query ==
[Jul 05, 01:20:25] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.1269,  0.0868,  0.0588,  ..., -0.0978,  0.0423,  0.0024],
        [ 0.0237,  0.0695,  0.0303,  ..., -0.0671, -0.0818, -0.0653],
        [ 0.0626,  0.0780,  0.0324,  ...,  0.0213, -0.0027,  0.1598],
        ...,
        [ 0.1314,  0.0969,  0.0671,  ..., -0.1059,  0.0561, -0.0338],
        [ 0.0398,  0.0168,  0.0476,  ..., -0.0210, -0.0167, -0.0480],
        [ 0.0871,  0.0832,  0.0208,  ..., -0.0546,  0.0003,  0.0225]],
       grad_fn=<SelectBackward0>)
[Jul 05, 01:20:25] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2613e-02, -3.6824e-03, -1.0773e-02,  ..., -1.3896e-02,
          1.2244e-02, -8.2326e-03],
        [-3.4988e-03,  3.9623e-03, -2.4905e-02,  ..., -2.6214e-02,
          4.0837e-03, -8.0812e-03],
        [-1.1158e-02, -2.1660e-05,  1.5019e-02,  ..., -1.1310e-02,
         -9.3691e-03, -2.2202e-02],
        ...,
        [-1.0295e-02,  3.2577e-02,  3.3341

Next we train the the teacher model, and save its location in the `teacher_model_fn`variable.

When training this model we use data with the same passages as for the student model, but with English translations of the queries.

In [7]:
text_triples_en_fn = os.path.join(test_files_location, "xorqa.train_ir_negs_5_poss_1_001pct_at_0pct_en.tsv")
args_dict = {
                'root': output_dir,
                'experiment': 'test_training_teacher',
                'triples': text_triples_en_fn,
                'model_type': model_type,
                'checkpoint': model_name_or_path,
                'maxsteps': 3,
                'bsize': 1,
                'accumsteps': 1,
                'amp': True,
                'epochs': 1,
                'rank': 0,
                'nranks': 1
            }

In [8]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)   
    teacher_model_fn = train(colBERTConfig, text_triples_en_fn, None, None)

{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "cosine",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta",
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta",
    "teacher_doc_maxlen": 180,
    "disti

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Jul 05, 01:21:15] 1 epochs of 5 examples
[Jul 05, 01:21:15] batch size: 1
[Jul 05, 01:21:15] maxsteps set to 3
[Jul 05, 01:21:15] start batch idx: 0
[Jul 05, 01:21:15] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Jul 05, 01:21:15] #> Input: $ Who maintained the throne for the longest time in China?, 		 True, 		 None
[Jul 05, 01:21:15] #> Output IDs: torch.Size([32]), tensor([    0,  9748, 40469, 76104,   297,    70,     6, 42294,    86,   100,
           70,  4989,   525,  1733,    23,  9098,    32,     2,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])
[Jul 05, 01:21:15] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[Jul 05, 01:21:15] #> XLMR DocTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Jul 05, 01:21:15] #> Input: $ Kangxi Emperor The Kangxi Emperor's reign of

/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 01:21:15] #>>>> before linear query ==
[Jul 05, 01:21:15] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.1266,  0.1302,  0.0628,  ..., -0.1106,  0.0519, -0.0265],
        [-0.0445,  0.0772,  0.0160,  ..., -0.0904, -0.0560, -0.0523],
        [-0.0128,  0.0967, -0.0064,  ..., -0.0320, -0.0303,  0.1300],
        ...,
        [ 0.1200,  0.1188, -0.0589,  ..., -0.2018, -0.0487, -0.0523],
        [ 0.0734,  0.0938, -0.0106,  ..., -0.1310, -0.0079,  0.0202],
        [ 0.1056,  0.1080, -0.0078,  ..., -0.0156, -0.0259,  0.0222]],
       grad_fn=<SelectBackward0>)
[Jul 05, 01:21:15] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2613e-02, -3.6824e-03, -1.0773e-02,  ..., -1.3896e-02,
          1.2244e-02, -8.2326e-03],
        [-3.4988e-03,  3.9623e-03, -2.4905e-02,  ..., -2.6214e-02,
          4.0837e-03, -8.0812e-03],
        [-1.1158e-02, -2.1660e-05,  1.5019e-02,  ..., -1.1310e-02,
         -9.3691e-03, -2.2202e-02],
        ...,
        [-1.0295e-02,  3.2577e-02,  3.3341

In the following two Knowledge Distillation (KD) steps, we will train the student model using 
1. parallel data contatining a) English and non-English passages for the student model, and b) English passages for the teacher model, 
2. parallel data containg a) non-English queries and English passages for the student model, and b) English queries and English passages for the student model.

First, we run Knowledge Distillation where the student learns teacher's token representations using the parallel English and non-English passage data from the following two files: 

a) English and non-English passages

b) English only passages:

In [9]:
parallel_non_en_fn = os.path.join(test_files_location, "7lan_notrim_triple_2ep.other.clean.h5")
parallel_en_fn = os.path.join(test_files_location, "7lan_notrim_triple_2ep.en.clean.h5")

Here is an example item from the mixed-language file:

In [10]:
data = pd.read_csv(parallel_non_en_fn, sep='\t', nrows=1, header=None)
display(HTML(data.to_html()))


,0,1,2
0,সে একটা জরুরি পুনর্বাসনের কাজ করছিল,She was right in the middle of an important piece of restoration .,She was right in the middle of an important piece of restoration .


Here are the parameters and command for the first Knowledge Distillation (KD) stage:

In [11]:
parallel_non_en_fn = os.path.join(test_files_location, "7lan_notrim_triple_2ep.other.clean.h5")
parallel_en_fn = os.path.join(test_files_location, "7lan_notrim_triple_2ep.en.clean.h5")

args_dict = {
    'root': output_dir,
    'experiment': 'test_training',
    'model_type': model_type,
    'checkpoint': student_model_fn + '-LAST.dnn',
    'distill_query_passage_separately': True,
    'teacher_model_type': model_type,
    'teacher_checkpoint': teacher_model_fn + '-LAST.dnn',
    'triples': parallel_non_en_fn,
    'teacher_triples': parallel_en_fn,
    'maxsteps': 3,
    'bsize': 1,
    'accumsteps': 1,
    'amp': True,
    'epochs': 1,
    'rank': 0,
    'nranks': 1
}

with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    stage_1_student_model_fn = train(colBERTConfig, parallel_non_en_fn, None, None)


{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "cosine",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta",
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta",
    "teacher_doc_maxlen": 180,
    "disti

/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 01:22:56] #>>>> before linear doc ==
[Jul 05, 01:22:56] #>>>>> D: torch.Size([18, 768]), tensor([[ 0.2977,  0.0828,  0.1011,  ..., -0.2781,  0.2468, -0.0562],
        [-0.0140,  0.0489, -0.0350,  ..., -0.0239, -0.0763, -0.0106],
        [-0.0233,  0.0752, -0.0080,  ..., -0.0425,  0.0524, -0.0914],
        ...,
        [-0.1848,  0.3038, -0.1414,  ...,  0.1956,  0.2507, -0.2537],
        [ 0.0078,  0.0946, -0.0887,  ..., -0.0424,  0.0233,  0.0098],
        [ 1.0042, -0.3377, -0.5804,  ..., -1.7804, -0.1052, -0.0873]],
       grad_fn=<SelectBackward0>)
[Jul 05, 01:22:56] #>>>>> self.linear doc : Parameter containing:
tensor([[-1.2618e-02, -3.6898e-03, -1.0781e-02,  ..., -1.3904e-02,
          1.2236e-02, -8.2367e-03],
        [-3.4999e-03,  3.9670e-03, -2.4897e-02,  ..., -2.6206e-02,
          4.0920e-03, -8.0773e-03],
        [-1.1149e-02, -2.6172e-05,  1.5013e-02,  ..., -1.1318e-02,
         -9.3766e-03, -2.2203e-02],
        ...,
        [-1.0288e-02,  3.2584e-02,  3.3274e-03

Next, we run Knowledge Distillation where the student learns teacher's relavance prediction score using parallel data containing:

a) non-English queries and English passages for the student model

b) English queries and English passages for the teacher model.

In [12]:
args_dict = {
    'root': output_dir,
    'experiment': 'test_training',
    'model_type': model_type,
    'checkpoint': stage_1_student_model_fn + '-LAST.dnn',
    'distill_query_passage_separately': False,
    'teacher_model_type': model_type,
    'teacher_checkpoint': teacher_model_fn + '-LAST.dnn',
    'triples': text_triples_en_fn,
    'teacher_triples': text_triples_en_fn,
    'maxsteps': 3,
    'bsize': 1,
    'accumsteps': 1,
    'amp': True,
    'epochs': 1,
    'rank': 0,
    'nranks': 1
}

with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    stage_2_student_model_fn = train(colBERTConfig, text_triples_fn, None, None)


{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "cosine",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta",
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta",
    "teacher_doc_maxlen": 180,
    "disti

[Jul 05, 01:24:31] #> Output Mask: torch.Size([152]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[Jul 05, 01:24:31] #>>>> colbert query ==
[Jul 05, 01:24:31] #>>>>> input_ids: torch.Size([32]), tensor([     0,   9748,  24120,   1180,  13968, 211059,  83639,  76826,  78363,
         57104,    993, 161732,    697, 116932, 114150,     32,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])


/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 01:24:32] #>>>> before linear query ==
[Jul 05, 01:24:32] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.0021,  0.1106,  0.0932,  ..., -0.1402,  0.0623,  0.0103],
        [ 0.0619,  0.0754,  0.0656,  ...,  0.0309, -0.0597, -0.1390],
        [ 0.1029,  0.0737,  0.0074,  ...,  0.0126, -0.0065,  0.1239],
        ...,
        [ 0.1028,  0.0929,  0.0215,  ..., -0.1302,  0.0043, -0.0321],
        [ 0.2147,  0.0945, -0.0322,  ..., -0.3424, -0.0917,  0.0465],
        [ 0.3043,  0.2025, -0.1540,  ..., -0.6458, -0.1479,  0.1297]],
       grad_fn=<SelectBackward0>)
[Jul 05, 01:24:32] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2610e-02, -3.6813e-03, -1.0783e-02,  ..., -1.3912e-02,
          1.2244e-02, -8.2292e-03],
        [-3.4997e-03,  3.9685e-03, -2.4898e-02,  ..., -2.6205e-02,
          4.0968e-03, -8.0693e-03],
        [-1.1151e-02, -2.6646e-05,  1.5022e-02,  ..., -1.1317e-02,
         -9.3691e-03, -2.2204e-02],
        ...,
        [-1.0289e-02,  3.2587e-02,  3.3357

[Jul 05, 01:24:35] #>>>> before linear doc ==
[Jul 05, 01:24:35] #>>>>> D: torch.Size([152, 768]), tensor([[ 0.1084,  0.0884,  0.0909,  ..., -0.0575,  0.0503, -0.0435],
        [-0.0571,  0.0447,  0.0464,  ..., -0.1166, -0.0801, -0.1011],
        [-0.1108,  0.1239, -0.0805,  ...,  0.3173,  0.0005,  0.2270],
        ...,
        [ 0.0990,  0.0783,  0.0172,  ..., -0.1854, -0.0516,  0.0065],
        [ 0.0990,  0.0783,  0.0172,  ..., -0.1854, -0.0516,  0.0065],
        [ 0.0990,  0.0783,  0.0172,  ..., -0.1854, -0.0516,  0.0065]])
[Jul 05, 01:24:35] #>>>>> self.linear doc : Parameter containing:
tensor([[-1.2614e-02, -3.6908e-03, -1.0778e-02,  ..., -1.3904e-02,
          1.2236e-02, -8.2380e-03],
        [-3.5076e-03,  3.9661e-03, -2.4907e-02,  ..., -2.6205e-02,
          4.0792e-03, -8.0751e-03],
        [-1.1165e-02, -1.3537e-05,  1.5018e-02,  ..., -1.1316e-02,
         -9.3630e-03, -2.2196e-02],
        ...,
        [-1.0294e-02,  3.2585e-02,  3.3258e-03,  ...,  3.1875e-02,
          3.

## Indexing
Next, we will index a collection of documents, using model representaion from the previous step. 
The collection is a TSV file, containing each document's ID, title, and text.

In [13]:
collection_fn = os.path.join(test_files_location, "xorqa.train_ir_001pct_at_0_pct_collection_fornum.tsv")

Here is an example document:

In [14]:
data = pd.read_csv(collection_fn, sep='\t', header=0, nrows=1)
#data = pd.read_csv(collection_fn, sep='\t', header=None, skiprows=3, nrows=1)
display(HTML(data.to_html()))

,id,text,title
0,1,"The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang.",Kangxi Emperor


Here are the indexer arguments:

In [15]:
args_dict = {
                'root': os.path.join(output_dir,'test_indexing'),
                'experiment': 'test_indexing',
                'model_type': model_type,
                'checkpoint': stage_2_student_model_fn,
                'collection': collection_fn,
                'index_root': os.path.join(output_dir, 'test_indexing', 'indexes'),
                'index_name': 'index_name',
                'doc_maxlen': 180,
                'num_partitions_max': 2,
                'kmeans_niters': 1,
                'nway': 1,
                'rank': 0,
                'nranks': 1,
                'amp': True
            }

Here we run the indexer:

In [16]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    create_directory(colBERTConfig.index_path_)
    encode(colBERTConfig, collection_fn, None, None)




[Jul 05, 01:25:58] #> Creating directory /tmp/tmp8pcpexf8/output_dir/test_indexing/indexes/index_name 


{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 1,
    "num_partitions_max": 2,
    "similarity": "cosine",
    "bsize": 32,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 500000,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 1,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 10,
    "input_arguments": {},
    "model_type": "xlm-roberta",
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher

/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 01:26:10] #>>>> before linear doc ==
[Jul 05, 01:26:10] #>>>>> D: torch.Size([180, 768]), tensor([[ 0.0670,  0.0966,  0.0534,  ..., -0.1338,  0.0548,  0.0021],
        [ 0.0021,  0.0295,  0.0277,  ..., -0.1455,  0.0118,  0.1002],
        [-0.0445,  0.0938, -0.0323,  ..., -0.2988, -0.0242,  0.0921],
        ...,
        [-0.0520,  0.0596,  0.0052,  ..., -0.1677, -0.0398,  0.0025],
        [-0.0520,  0.0596,  0.0052,  ..., -0.1677, -0.0398,  0.0025],
        [-0.0520,  0.0596,  0.0052,  ..., -0.1677, -0.0398,  0.0025]])
[Jul 05, 01:26:10] #>>>>> self.linear doc : Parameter containing:
tensor([[-1.2606e-02, -3.6783e-03, -1.0787e-02,  ..., -1.3916e-02,
          1.2240e-02, -8.2368e-03],
        [-3.5033e-03,  3.9692e-03, -2.4890e-02,  ..., -2.6206e-02,
          4.1044e-03, -8.0725e-03],
        [-1.1151e-02, -3.5383e-05,  1.5014e-02,  ..., -1.1308e-02,
         -9.3683e-03, -2.2202e-02],
        ...,
        [-1.0285e-02,  3.2592e-02,  3.3408e-03,  ...,  3.1879e-02,
          3.

0it [00:00, ?it/s]

[Jul 05, 01:26:10] [0] 		 #> Encoding 7 passages..
[Jul 05, 01:26:11] [0] 		 #> Saving chunk 0: 	 7 passages and 1,222 embeddings. From #0 onward.


1it [00:01,  1.75s/it]

[Jul 05, 01:26:11] offset: 0
[Jul 05, 01:26:11] chunk codes size(0): 1222
[Jul 05, 01:26:11] codes size(0): 1222
[Jul 05, 01:26:11] codes size(): torch.Size([1222])
[Jul 05, 01:26:11] >>>>partition.size(0): 2
[Jul 05, 01:26:11] >>>>num_partition: 2
[Jul 05, 01:26:11] #> Optimizing IVF to store map from centroids to list of pids..
[Jul 05, 01:26:11] #> Building the emb2pid mapping..
[Jul 05, 01:26:11] len(emb2pid) = 1222



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 6227.62it/s]

[Jul 05, 01:26:11] #> Saved optimized IVF to /tmp/tmp8pcpexf8/output_dir/test_indexing/indexes/index_name/ivf.pid.pt
[Jul 05, 01:26:11] [0] 		 #> Saving the indexing metadata to /tmp/tmp8pcpexf8/output_dir/test_indexing/indexes/index_name/metadata.json ..


The resulting index files are in `output_dir/test_indexing/indexes/index_name/metadata.json`

## Search
Next, we use the trained model and the index to search the collection, using queries from a TSV query file.

In [17]:
queries_fn = os.path.join(test_files_location, "xorqa.train_ir_001pct_at_0_pct_queries_fornum.tsv")

Here are the search arguments:

In [18]:
args_dict = {
                'root': output_dir,
                'experiment': 'test_indexing' ,
                'model_type': model_type,
                'checkpoint': stage_2_student_model_fn,
                'model_type': model_type,
                'index_location': os.path.join(output_dir, 'test_indexing', 'indexes', 'index_name'),
                'queries': queries_fn,
                'bsize': 1,
                'topK': 1,
                'nway': 1,
                'rank': 0,
                'nranks': 1,
                'amp': True,
            }

Here we initalize and run the searcher:

In [19]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    searcher = Searcher(args_dict['index_location'], checkpoint=args_dict['checkpoint'], config=colBERTConfig)
    rankings = searcher.search_all(args_dict['queries'], args_dict['topK'])

[Jul 05, 01:26:51] #> base_config.py from_path /tmp/tmp8pcpexf8/output_dir/test_indexing/indexes/index_name/metadata.json
[Jul 05, 01:26:51] #> base_config.py from_path args loaded! 
[Jul 05, 01:26:51] #> base_config.py from_path args replaced ! 
[Jul 05, 01:26:51] #> base_config.py load_from_checkpoint /tmp/tmp8pcpexf8/output_dir/test_training/2023-07/05/01.19.46/checkpoints/colbert
[Jul 05, 01:26:51] #> base_config.py load_from_checkpoint /tmp/tmp8pcpexf8/output_dir/test_training/2023-07/05/01.19.46/checkpoints/colbert/artifact.metadata
[Jul 05, 01:26:51] #> base_config.py from_path /tmp/tmp8pcpexf8/output_dir/test_training/2023-07/05/01.19.46/checkpoints/colbert/artifact.metadata
[Jul 05, 01:26:51] #> base_config.py from_path args loaded! 
[Jul 05, 01:26:51] #>>>>> at ColBERT name (model name) : /tmp/tmp8pcpexf8/output_dir/test_training/2023-07/05/01.19.46/checkpoints/colbert
[Jul 05, 01:26:51] #>>>>> at BaseColBERT name (model name) : /tmp/tmp8pcpexf8/output_dir/test_training/2023-

/dccstor/cssblr/rbhat/.envs/pqa/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Jul 05, 01:27:04] #>>>> before linear query ==
[Jul 05, 01:27:04] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.1501,  0.0966,  0.0610,  ..., -0.1078,  0.0462, -0.0130],
        [ 0.0766,  0.0557,  0.0596,  ..., -0.0656, -0.1207, -0.0641],
        [ 0.1129,  0.0510,  0.0124,  ...,  0.0046, -0.0049,  0.2024],
        ...,
        [ 0.1389,  0.0867, -0.0129,  ..., -0.2225, -0.0461,  0.0348],
        [ 0.1389,  0.0867, -0.0129,  ..., -0.2225, -0.0461,  0.0348],
        [ 0.1389,  0.0867, -0.0129,  ..., -0.2225, -0.0461,  0.0348]])
[Jul 05, 01:27:04] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2606e-02, -3.6783e-03, -1.0787e-02,  ..., -1.3916e-02,
          1.2240e-02, -8.2368e-03],
        [-3.5033e-03,  3.9692e-03, -2.4890e-02,  ..., -2.6206e-02,
          4.1044e-03, -8.0725e-03],
        [-1.1151e-02, -3.5383e-05,  1.5014e-02,  ..., -1.1308e-02,
         -9.3683e-03, -2.2202e-02],
        ...,
        [-1.0285e-02,  3.2592e-02,  3.3408e-03,  ...,  3.1879e-02,
         

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.21it/s]


Here is the search result for our query, containing [query_id, document_id, rank, score]:

In [20]:
rankings.flat_ranking[0]

('-7239279093922981232', 1, 1, 30.22616195678711)

Here is the text of the query record, contataing the ID and text of the query:

In [21]:
with open(queries_fn, 'r') as f:
    for line in f.readlines():
        if str(rankings.flat_ranking[0][0]) == line.split()[0]:
            print(line)

-7239279093922981232	중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?



English translation: `Who maintained the throne for the longest time in China?`

Here is the top retrieved document:

In [22]:
with open(collection_fn, 'r') as f:
    for line in f.readlines():
        if str(rankings.flat_ranking[0][1]) == line.split()[0]:
            print(line)

1	"The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang."	Kangxi Emperor

